# Selector with ML
### Generate and collect data on quantum circuits of sequentially increasing complexities

### Imports

In [2]:
from qiskit import *
import pandas as pd
from qiskit import transpile
import random
import csv
import numpy as np
from qiskit.providers.aer import AerSimulator
import time
import itertools
import pickle
import random
import progressbar
import pathlib
from IPython.display import clear_output
from qiskit.tools.monitor import job_monitor

# Import IBMQ and Aer from qiskit
from qiskit import IBMQ, Aer

# Import NoiseModel from qiskit.providers.aer.noise
from qiskit.providers.aer.noise import NoiseModel

# Load the IBMQ account
provider = IBMQ.load_account()

# Get the provider from IBMQ with specified hub, group, and project
provider = IBMQ.get_provider(hub='ibm-q-research-2', group='los-gatos-hs-1', project='main')

# Get the backend named 'ibm_perth' from the provider
backend = provider.get_backend('ibm_perth')

# Set the number of qubits to 7
qubitCount = 7

# Create a noise model from the backend
noise_model = NoiseModel.from_backend(backend)

# Get the coupling map from the backend configuration
coupling_map = backend.configuration().coupling_map

# Get the basis gates from the noise model
basis_gates = noise_model.basis_gates

# Define a list of gate names
Gates = ['rz', 'rx', 'ry', 'sx', 'x', 'y', 'z', 'h', 'cx', 'swap']

# Define a list of gate costs
gateCosts = [1, 1, 1, 1, 1, 1, 1, 2, 5, 11]

# Define fields for the first set of data
fields1 = ['Cost', 'Gate Count', 'Optimization Level', 'Runtime', 'Transpile Time']

# Define fields for the second set of data
fields2 = ['ActCost', 'Act Gate Count', 'sim Cost 0', 'sim Gate Count 0', 'sim Runtime 0', 'sim Transpile Time 0',
           'sim Cost 1', 'sim Gate Count 1', 'sim Runtime 1', 'sim Transpile Time 1',
           'sim Cost 2', 'sim Gate Count 2', 'sim Runtime 2', 'sim Transpile Time 2',
           'sim Cost 3', 'sim Gate Count 3', 'sim Runtime 3', 'sim Transpile Time 3',
           'Cost 0', 'Gate Count 0', 'Runtime 0', 'Transpile Time 0', 'Qubits 0',
           'Cost 1', 'Gate Count 1', 'Runtime 1', 'Transpile Time 1', 'Qubits 1',
           'Cost 2', 'Gate Count 2', 'Runtime 2', 'Transpile Time 2', 'Qubits 2',
           'Cost 3', 'Gate Count 3', 'Runtime 3', 'Transpile Time 3', 'Qubits 3',
          ]

# Load the MLModel from the file 'PerthSimRegModel.sav'
filename = 'MLModels//PerthSimRegModel.sav'
reg = pickle.load(open(filename, 'rb'))


C:\Users\taren\AppData\Local\Temp\ipykernel_7320\1820962702.py:19: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  provider = IBMQ.load_account()
C:\Users\taren\AppData\Local\Temp\ipykernel_7320\1820962702.py:19: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  provider = IBMQ.load_account()
C:\Users\taren\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying

### Functions

In [3]:

def QCtoDF(qc):
    string = qc.qasm()
    circuit = string.split(';')
    circuit = circuit[3:]
    circuit.pop(len(circuit)-1)
    #print(circuit)
    with open(r"gatesTemp.csv", 'w', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(['Gate', 'Qubit'])
        for i in range(len(circuit)):            
            circuit[i] = circuit[i].replace("\n", '')
            #print(circuit[0][1])
            temp = circuit[i].split(' ')
            #temp[1] = temp[1].split(',')
            for j in range(len(temp[1])):
                 temp[1] = temp[1].replace("q[", '')
                 temp[1] = temp[1].replace("]", '')
            if temp[0] != 'measure' and temp[0] != 'barrier' and temp[0] != 'creg':
                writer.writerow(temp)
    df = pd.DataFrame(pd.read_csv(r"gatesTemp.csv"))
    return df

def unique(list1):
  
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)

def depthFinder(df):
    qubits = df['Qubit'].value_counts()
    qubits = dict(qubits)
    lst = list(qubits.items())
    #get depth
    if len(lst) > 1:
        qnum = []
        for j in range(len(lst)):
            strList = str(lst[j][0]).split(',')
            qnum.append(strList[0])
        qnum = unique(qnum)
        counts = []
        for num in qnum:
            for j in range(len(lst)):
                strList = str(lst[j][0]).split(',')
                if strList[0] == num:
                    for times in range(lst[j][1]):
                        counts.append(strList[0])
        return len(counts) 
    elif len(lst) > 0:
        return(lst[0][1])
    else:
        return(0)
    
    
def ComplexityFinder(qc):
    df = QCtoDF(qc)
    depth = depthFinder(df)
    #Get Cost
    cost = 0
    for i in df['Gate']:
        if i in Gates:
            cost += int(gateCosts[Gates.index(i)])
        else:
            cost += int(gateCosts[Gates.index(i[:2])])
        
    return cost, depth

def binary_search(cost, name):  
    low = 0  
    high = int((np.math.factorial(4 + name - 1))/(np.math.factorial(name)*(np.math.factorial(3)))) - 1
    mid = 0  
  
    while low <= high:  
        # for get integer result   
        mid = (high + low) // 2  
        # print("Name", name)
        try:
            dfFull = pd.read_csv("D:/Documents/Combinations/combo" + str(name) + ".csv", skiprows = mid, nrows=1)
        except:
            with open(r"ErrorLog.csv", 'w', newline='', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow([name])
                return -1
        # Check if n is present at mid   
        if dfFull.loc[0][0] < cost:  
            low = mid + 1  
  
        # If n is greater, compare to the right of mid   
    
        elif dfFull.loc[0][0] > cost:  
            high = mid - 1  
  
        # If n is smaller, compared to the left of mid  
        else:  
            return mid  
  
            # element was not present in the list, return -1  
    return -1  
# binary_search(25, 20)

def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
def check(x):
    return x and [x[0]]*len(x) == x

In [ ]:
# Selector Function
def selector(qc):
    start = time.time()
    runs = 1
    
    # Initialize lists to store results for different optimization levels
    TT = []
    Cost = []
    GC = []
    RT = []
    
    # Optimize the circuit and calculate metrics for optimization level 3
    for run in range(runs):
        start_time = time.time()
        tqc3 = transpile(qc, backend, optimization_level=3)
        TT3 = time.time() - start_time
        TT.append(TT3)
        Cost3, GC3 = ComplexityFinder(tqc3)
        Cost.append(Cost3)
        GC.append(GC3)
        RT3 = reg.predict([[Cost3, GC3, 3, TT3]])
        RT.append(RT3)
    TT3 = Average(TT)
    Cost3 = Average(Cost)
    GC3 = Average(GC)
    RT3 = Average(RT)
    s3 = [Cost3, GC3, RT3[0], TT3]

    # Optimize the circuit and calculate metrics for optimization level 2
    TT = []
    Cost = []
    GC = []
    RT = []
    for run in range(runs):
        start_time = time.time()
        tqc2 = transpile(qc, backend, optimization_level=2)
        TT2 = time.time() - start_time
        TT.append(TT2)
        Cost2, GC2 = ComplexityFinder(tqc2)
        Cost.append(Cost2)
        GC.append(GC2)
        RT2 = reg.predict([[Cost2, GC2, 2, TT2]])
        RT.append(RT2)
    TT2 = Average(TT)
    Cost2 = Average(Cost)
    GC2 = Average(GC)
    RT2 = Average(RT)
    s2 = [Cost2, GC2, RT2[0], TT2]

    # Optimize the circuit and calculate metrics for optimization level 1
    TT = []
    Cost = []
    GC = []
    RT = []
    for run in range(runs):
        start_time = time.time()
        tqc1 = transpile(qc, backend, optimization_level=1)
        TT1 = time.time() - start_time
        TT.append(TT1)
        Cost1, GC1 = ComplexityFinder(tqc1)
        Cost.append(Cost1)
        GC.append(GC1)
        RT1 = reg.predict([[Cost1, GC1, 1, TT1]])
        RT.append(RT1)
    TT1 = Average(TT)
    Cost1 = Average(Cost)
    GC1 = Average(GC)
    RT1 = Average(RT)
    s1 = [Cost1, GC1, RT1[0], TT1]

    # Optimize the circuit and calculate metrics for optimization level 0
    TT = []
    Cost = []
    GC = []
    RT = []
    for run in range(runs):
        start_time = time.time()
        tqc0 = transpile(qc, backend, optimization_level=0)
        TT0 = time.time() - start_time
        TT.append(TT0)
        Cost0, GC0 = ComplexityFinder(tqc0)
        Cost.append(Cost0)
        GC.append(GC0)
        RT0 = reg.predict([[Cost0, GC0, 0, TT0]])
        RT.append(RT0)
    TT0 = Average(TT)
    Cost0 = Average(Cost)
    GC0 = Average(GC)
    RT0 = Average(RT)
    s0 = [Cost0, GC0, RT0[0], TT0]
    
    # Calculate the runtime of each optimization level for this qc
    tqc3RT = RT3[0] + TT3
    tqc2RT = RT2[0] + TT2
    tqc1RT = RT1[0] + TT1
    tqc0RT = RT0[0] + TT0
    RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]
    GCs = [GC0, GC1, GC2, GC3]
    Costs = [Cost0, Cost1, Cost2, Cost3]

    # Cascade choice
    optLvls = [0, 1, 2, 3]
    worst = RTs.index(max(RTs))
    optLvls.pop(worst)
    RTs.pop(worst)
    GCs.pop(worst)
    Costs.pop(worst)

    # If all items are identical, choose the one with the maximum runtime
    if check(Costs):
        worst = RTs.index(max(RTs))
    else:
        worst = Costs.index(max(Costs))
    optLvls.pop(worst)
    RTs.pop(worst)
    GCs.pop(worst)
    Costs.pop(worst)
    
    # If all items are identical, choose the one with the maximum gate count
    if check(GCs):
        worst = RTs.index(max(RTs))
    else:
        worst = GCs.index(max(GCs))
    optLvls.pop(worst)
    
    # Choose the best optimization level from the remaining options
    best = optLvls[0]
    print("old best: ", best)
    
    # Check if any RTs are too close to be distinguished
    RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]
    GCs = [GC0, GC1, GC2, GC3]
    Costs = [Cost0, Cost1, Cost2, Cost3]
    alternatives = []
    for i in range(len(RTs)):
        if i != best:
            # If the difference in runtime is within a threshold and gate count and cost are lower,
            # consider it as an alternative to the current best optimization level
            if (RTs[i] - RTs[best]) / RTs[best] * 100 <= 30 and GCs[i] < GCs[best] and Costs[i] < Costs[best]:
                best = i
            elif (RTs[i] - RTs[best]) / RTs[best] * 100 <= 5 and GCs[i] <= GCs[best] and Costs[i] <= Costs[best]:
                alternatives.append(i)
    
    print("RTs: ", RTs)
    print("csv GCs: ", GCs)
    print("Costs: ", Costs)
    return best, alternatives, s3, s2, s1, s0, time.time() - start


### Circuit Generation

In [ ]:
simTimes = []
selTimes = []
simVal = []
simAlts = []
controlVal = []
controlAlts = []
final = []

extFinal = []

shots = 1024
runs = 1

maxComboLength = 290

# percent = 5
for cost in range(0, 50 + 1):
# for rep in range(1, 200):
#     cost = random.randint(1, 100 + 1)
    #find all combinations of gates to = cost, include entanglement as soon as cost reaches large enough number
    #max GC == cost
    if cost >= maxComboLength:
        break
    for totalGates in range(1, cost + 1):
        # totalGates = random.randint(1, cost + 1)
        clear_output(wait=True)
        idx = binary_search(cost, totalGates)
        # print("Rep", rep)
        print("Cost:", cost)
        print("Total Number of Gates:", totalGates)
        #No possible combination
        if idx != -1:
            combination = []
            df = pd.read_csv(r"D:/Documents/Combinations/combo" + str(totalGates) + ".csv", skiprows = idx, nrows=1)
            for i in df.loc[0]:
                if i in Gates:
                    combination.append(i)

            # print("Cost:", cost)
            # print("Total Number of Gates:", totalGates)
            qc = QuantumCircuit(qubitCount)
            if combination != []:
                comboTemp = combination
                random.shuffle(comboTemp)
                combo = []
                #Replace all x gates with x, y, or z
                for gate in comboTemp:
                    if gate == 'x':
                        combo.append(random.choice(['x', 'y', 'z']))
                    else:
                        combo.append(gate)
                #append all to first qubit until GC is met
                #the rest are distributed randomly to the others 
                for gateIdx in range(len(combo)):
                    if combo[gateIdx] == 'cx' or combo[gateIdx] == 'swap':
                        randint1 = random.randint(0, qubitCount - 1)
                        randint2 = random.randint(0, qubitCount - 1)
                        while randint2 == randint1:
                            randint2 = random.randint(0, qubitCount - 1)
                        getattr(qc, combo[gateIdx])(randint1, randint2)
                    else:
                        randint1 = random.randint(0, qubitCount - 1)
                        getattr(qc, combo[gateIdx])(randint1)
                # listToAppend.append(cost)
                # listToAppend.append(totalGates)

                qc.measure_all()
                print(qc)

                listToAppendExt = [cost, totalGates]
                alternatives = []
                testBest, alternatives, s3, s2, s1, s0, selTime = selector(qc)
                selTimes.append(selTime)
                print("Chosen Optimization Level: ", testBest)
                print("Choosen Alternatives: ", alternatives)
                simVal.append(testBest)
                simAlts.append(alternatives)
                listToAppendExt.extend(s0)
                listToAppendExt.extend(s1)
                listToAppendExt.extend(s2)
                listToAppendExt.extend(s3)

                # Simulator CODE
                start = time.time()
                RTTimes = []
                TTTimes = []
                GCs = []
                costs = []

                tempRT = []
                tempTT = []
                tempGC= []
                tempC = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 0)
                    tempTT.append(time.time() - start_time)
                    tqcCost, tqcGC = ComplexityFinder(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model, shots = shots).result()
                    # job = execute(tqc, backend=backend)
                    # job_monitor(job)
                    # result = job.result()
                    tempRT.append(result.time_taken)
                    tempGC.append(tqcGC)
                    tempC.append(tqcCost)
                RTTimes.append(Average(tempRT))
                TTTimes.append(Average(tempTT))
                GCs.append(Average(tempGC))
                costs.append(Average(tempC))
                qubitdf = QCtoDF(tqc)
                newQubits = qubitdf['Qubit'].value_counts()
                newQubits = dict(newQubits)
                listToAppend = [Average(tempC), Average(tempGC), 0, Average(tempRT), Average(tempTT)]
                final.append(listToAppend)
                listToAppendExt.extend([Average(tempC), Average(tempGC), Average(tempRT), Average(tempTT), newQubits])


                tempRT = []
                tempTT = []
                tempGC = []
                tempC = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 1)
                    tempTT.append(time.time() - start_time)
                    tqcCost, tqcGC = ComplexityFinder(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model, shots = shots).result()
                    # job = execute(tqc, backend=backend)
                    # job_monitor(job)
                    # result = job.result()
                    tempRT.append(result.time_taken)
                    tempGC.append(tqcGC)
                    tempC.append(tqcCost)
                RTTimes.append(Average(tempRT))
                TTTimes.append(Average(tempTT))
                GCs.append(Average(tempGC))
                costs.append(Average(tempC))
                qubitdf = QCtoDF(tqc)
                newQubits = qubitdf['Qubit'].value_counts()
                newQubits = dict(newQubits)
                listToAppend = [Average(tempC), Average(tempGC), 1, Average(tempRT), Average(tempTT)]
                final.append(listToAppend)
                listToAppendExt.extend([Average(tempC), Average(tempGC), Average(tempRT), Average(tempTT), newQubits])

                tempRT = []
                tempTT = []
                tempGC = []
                tempC = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 2)
                    tempTT.append(time.time() - start_time)
                    tqcCost, tqcGC = ComplexityFinder(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model, shots = shots).result()
                    # job = execute(tqc, backend=backend)
                    # job_monitor(job)
                    # result = job.result()
                    tempRT.append(result.time_taken)
                    tempGC.append(tqcGC)
                    tempC.append(tqcCost)
                RTTimes.append(Average(tempRT))
                TTTimes.append(Average(tempTT))
                GCs.append(Average(tempGC))
                costs.append(Average(tempC))
                qubitdf = QCtoDF(tqc)
                newQubits = qubitdf['Qubit'].value_counts()
                newQubits = dict(newQubits)
                listToAppend = [Average(tempC), Average(tempGC), 2, Average(tempRT), Average(tempTT)]
                final.append(listToAppend)
                listToAppendExt.extend([Average(tempC), Average(tempGC), Average(tempRT), Average(tempTT), newQubits])

                tempRT = []
                tempTT = []
                tempGC = []
                tempC = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 3)
                    tempTT.append(time.time() - start_time)
                    tqcCost, tqcGC = ComplexityFinder(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model, shots = shots).result()
                    # job = execute(tqc, backend=backend)
                    # job_monitor(job)
                    # result = job.result()
                    tempRT.append(result.time_taken)
                    tempGC.append(tqcGC)
                    tempC.append(tqcCost)
                RTTimes.append(Average(tempRT))
                TTTimes.append(Average(tempTT))
                GCs.append(Average(tempGC))
                costs.append(Average(tempC))
                qubitdf = QCtoDF(tqc)
                newQubits = qubitdf['Qubit'].value_counts()
                newQubits = dict(newQubits)
                listToAppend = [Average(tempC), Average(tempGC), 3, Average(tempRT), Average(tempTT)]
                final.append(listToAppend)
                listToAppendExt.extend([Average(tempC), Average(tempGC), Average(tempRT), Average(tempTT), newQubits])

                extFinal.append(listToAppendExt)

                tqc3RT = float(RTTimes[3]) + float(TTTimes[3])
                tqc2RT = float(RTTimes[2]) + float(TTTimes[2])
                tqc1RT = float(RTTimes[1]) + float(TTTimes[1])
                tqc0RT = float(RTTimes[0]) + float(TTTimes[0])
                RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]
                # best = RTs.index(min(tqc3RT, tqc2RT, tqc1RT, tqc0RT))


                print("Simulated Times: ", RTs)
                # print("Accuracies: ", accuracies)
                print("Simulated GCs: ", GCs)
                print("Simulated Costs: ", costs) 

                RTs2 = [RTs[0], RTs[1], RTs[2], RTs[3]]
                GCs2 = [GCs[0], GCs[1], GCs[2], GCs[3]]
                costs2 = [costs[0], costs[1], costs[2], costs[3]]

                optLvls = [0, 1, 2, 3]
                worst = RTs.index(max(RTs))
                optLvls.pop(worst)
                RTs.pop(worst)
                GCs.pop(worst)
                costs.pop(worst)
                #all items are identical
                if check(costs):
                    worst = RTs.index(max(RTs))
                else:
                      worst = costs.index(max(costs))
                optLvls.pop(worst)
                RTs.pop(worst)
                GCs.pop(worst)
                costs.pop(worst)
                if check(GCs):
                    worst = RTs.index(max(RTs))
                else:
                    worst = GCs.index(max(GCs))
                optLvls.pop(worst)
                best = optLvls[0]
                print("old best: ", best)
                #Check if any RTs are too close to be distinguished
                alternatives = []
                for i in range(len(RTs2)):
                    if i != best:
                        # print(i)
                        # print((RTs2[i] - RTs2[best]) / RTs2[best] *100)
                        if (RTs2[i] - RTs2[best]) / RTs2[best] *100 <= 30 and GCs2[i] < GCs2[best] and costs2[i] < costs2[best]:
                            best = i
                        elif (RTs2[i] - RTs2[best]) / RTs2[best] *100 <= 5 and GCs2[i] <= GCs2[best] and costs2[i] <= costs2[best]:
                            alternatives.append(i)
                # print("RTs: ", RTs)
                print("Chosen Sim Optimization Level: ", best)
                print("Alternative Optimization Lvls: ", alternatives)
                simTimes.append(time.time() - start)
                controlVal.append(best)
                controlAlts.append(alternatives)
                print("--------------------------------------------------------------")

Cost: 1
Total Number of Gates: 1
              ░ ┌─┐                  
   q_0: ──────░─┤M├──────────────────
        ┌───┐ ░ └╥┘┌─┐               
   q_1: ┤ Z ├─░──╫─┤M├───────────────
        └───┘ ░  ║ └╥┘┌─┐            
   q_2: ──────░──╫──╫─┤M├────────────
              ░  ║  ║ └╥┘┌─┐         
   q_3: ──────░──╫──╫──╫─┤M├─────────
              ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ──────░──╫──╫──╫──╫─┤M├──────
              ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ──────░──╫──╫──╫──╫──╫─┤M├───
              ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_6: ──────░──╫──╫──╫──╫──╫──╫─┤M├
              ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/═════════╩══╩══╩══╩══╩══╩══╩═
                 0  1  2  3  4  5  6 
old best:  2
RTs:  [0.11275383691834864, 0.11086015543556543, 0.10423185466721362, 0.20233371298224959]
csv GCs:  [1.0, 1.0, 1.0, 1.0]
Costs:  [1.0, 1.0, 1.0, 1.0]
Chosen Optimization Level:  2
Choosen Alternatives:  []
Job Status: job is queued (None)


KeyboardInterrupt



### Save Data

In [10]:
with open(r"QMachineData/PerthDataSimFull3.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(fields2)
    writer.writerows(extFinal)

In [16]:
with open("QMachineData/ListData/simValPerth", "wb") as fp:   #Pickling
    pickle.dump(simVal, fp)
with open("QMachineData/ListData/simAltsPerth", "wb") as fp:   #Pickling
    pickle.dump(simAlts, fp)
with open("QMachineData/ListData/controlValPerth", "wb") as fp:   #Pickling
    pickle.dump(controlVal, fp)
with open("QMachineData/ListData/controlAltsPerth", "wb") as fp:   #Pickling
    pickle.dump(controlAlts, fp)

In [17]:
with open(r"QMachineData/PerthDataRealTemp.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(fields1)
    writer.writerows(final)
    
df = pd.read_csv("QMachineData/PerthDataRealTemp.csv")
df.to_csv("QMachineData/PerthDataReal.csv", mode='a', index=False, header=False)

#Extended data
with open(r"QMachineData/PerthDataRealTemp.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(fields2)
    writer.writerows(extFinal)
    
df = pd.read_csv("QMachineData/PerthDataRealTemp.csv")
df.to_csv("QMachineData/PerthDataRealExtended.csv", mode='a', index=False, header=False)


In [58]:
#Get Accuracy
#simVal is the array of selected optimization levels by my Selector function
#controlVal is the array of selected optimization levels by the control
correct = 0
key = []
for i in range(len(simVal)):
    point = False
    if simVal[i] == controlVal[i]:
        correct += 1
        key.append(1)
        point = True
    for controlAlt in controlAlts[i]:
        if simVal[i] == controlAlt and point == False:
            correct += 1
            key.append(1)
            point = True
            break
    for alt in simAlts[i]:
        if alt == controlVal[i] and point == False:
            correct += 1
            key.append(1)
            point = True
            break
    for alt in simAlts[i]:
        for controlAlt in controlAlts[i]:
            if alt == controlAlt and point == False:
                correct += 0.5
                point = True
                key.append(.5)
                break
    if point == False:
        key.append(0)

with open("QMachineData/ListData/key", "wb") as fp:   #Pickling
    pickle.dump(key, fp)

print("Accuracy: ", 100* (correct/(len(simVal)))) 

Accuracy:  85.48387096774194


In [ ]:
len(simVal)

13

In [20]:
t = 0

for i in range(len(selTimes)):
    t += selTimes[i]
print(t)
print("Time: ", t/(len(selTimes))) 

94.70192790031433
Time:  0.08358510847335775


In [21]:
t = 0

for i in range(len(simTimes)):
    t += simTimes[i]
print(t)
print("Time: ", t/(len(simTimes))) 

588.3540463447571
Time:  0.5192886552027864


In [10]:
simVal.pop(-1)

2

In [9]:
len(simAlts)

14

In [11]:
simAlts.pop(-1)

[1, 3]

In [21]:
len(controlVal)

171

In [12]:
len(controlAlts)

34

In [59]:
lengths = []
for i in controlAlts:
    lengths.append(len(i))

In [60]:
# importing the module
import collections

# using Counter to find frequency of elements
frequency = collections.Counter(lengths)

# printing the frequency
print(dict(frequency))

{1: 12, 2: 6, 0: 44}


In [61]:
avgSim = Average(simTimes)
avgSel = Average(selTimes)
print(avgSim)
print(avgSel)
print(avgAct)

print(((avgSim - avgSel)/avgSim) * 100)


0.5876283145719959
0.11628908111203101
3719.334634890923
80.21043604804319


In [54]:
job_id = '63e64d4b42f079c640fcd321'
job = backend.retrieve_job(job_id)

In [57]:
print(job.qobj())

QASM Qobj: d5069e88-741b-4fa2-8980-8c1f8384f962:
Config: {'init_qubits': True,
 'meas_level': 2,
 'memory': False,
 'memory_slots': 7,
 'n_qubits': 7,
 'parameter_binds': [],
 'parametric_pulses': ['gaussian',
                       'gaussian_square',
                       'gaussian_square_drag',
                       'drag',
                       'constant'],
 'rep_delay': 250.0,
 'shots': 4000,
 'use_measure_esp': True}
Header: {'backend_name': 'ibm_perth', 'backend_version': '1.1.41'}
Experiments:

QASM Experiment:
Header:
{'clbit_labels': [['meas', 0],
                  ['meas', 1],
                  ['meas', 2],
                  ['meas', 3],
                  ['meas', 4],
                  ['meas', 5],
                  ['meas', 6]],
 'creg_sizes': [['meas', 7]],
 'global_phase': 0.0,
 'memory_slots': 7,
 'metadata': {},
 'n_qubits': 7,
 'name': 'circuit-31131',
 'qreg_sizes': [['q', 7]],
 'qubit_labels': [['q', 0],
                  ['q', 1],
                  ['q', 2],
     

C:\Users\taren\AppData\Local\Temp\ipykernel_25196\1899523931.py:1: DeprecationWarning: The ``IBMQJob.qobj()`` method is deprecated and will be removed in a future release. You can now pass circuits to ``IBMQBackend.run()`` and use ``IBMQJob.circuits()``, ``IBMQJob.backend_options()``, and ``IBMQJob.header()`` to retrieve circuits, run configuration, and Qobj header, respectively.
  print(job.qobj())


# Old

In [36]:
#Import all processes necessary to run Circuit Generator
from qiskit import *
import pandas as pd
from qiskit import transpile
import random
import csv
import numpy as np
from qiskit.providers.aer import AerSimulator
import time
import itertools
import random
import progressbar
import pathlib
from IPython.display import clear_output
from qiskit.tools.monitor import job_monitor

from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
provider = IBMQ.load_account()
# IBMQ.load_account()
# provider = IBMQ.get_provider('ibm-q')
backend = provider.get_backend('ibm_oslo')

qubitCount = 7
runs = 10
dataFile = "QMachineData//OsloDataAVG4.csv"

noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

Gates = ['x', 'y', 'z', 'h', 'cx', 'swap']
gateCosts = [1, 1, 1, 2, 5, 11]
fields = ['cost', 'depth', 'tqc3TT', 'tqc3RT', 'tqc3Depth', 'tqc2TT', 'tqc2RT', 'tqc2Depth', 'tqc1TT', 'tqc1RT', 'tqc1Depth', 'tqc0TT', 'tqc0RT', 'tqc0Depth']

ibmqfactory.load_account:WARNING:2022-12-30 20:28:48,271: Credentials are already in use. The existing account in the session will be replaced.


### Functions

In [37]:

def QCtoDF(qc):
    string = qc.qasm()
    circuit = string.split(';')
    circuit = circuit[3:]
    circuit.pop(len(circuit)-1)
    #print(circuit)
    with open(r"gatesTemp.csv", 'w', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(['Gate', 'Qubit'])
        for i in range(len(circuit)):            
            circuit[i] = circuit[i].replace("\n", '')
            #print(circuit[0][1])
            temp = circuit[i].split(' ')
            #temp[1] = temp[1].split(',')
            for j in range(len(temp[1])):
                 temp[1] = temp[1].replace("q[", '')
                 temp[1] = temp[1].replace("]", '')
            if temp[0] != 'measure' and temp[0] != 'barrier' and temp[0] != 'creg':
                writer.writerow(temp)
    df = pd.DataFrame(pd.read_csv(r"gatesTemp.csv"))
    return df

def unique(list1):
  
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)

def depthFinder(df):
    qubits = df['Qubit'].value_counts()
    qubits = dict(qubits)
    lst = list(qubits.items())
    #get depth
    if len(lst) > 1:
        qnum = []
        for j in range(len(lst)):
            strList = str(lst[j][0]).split(',')
            qnum.append(strList[0])
        qnum = unique(qnum)
        counts = []
        for num in qnum:
            for j in range(len(lst)):
                strList = str(lst[j][0]).split(',')
                if strList[0] == num:
                    for times in range(lst[j][1]):
                        counts.append(strList[0])
        temp = []
        for num in qnum:
            temp.append(counts.count(num))
        mx = max(temp)
        return mx 
    elif len(lst) > 0:
        return(lst[0][1])
    else:
        return(0)
    
def ComplexityFinder(qc):
    df = QCtoDF(qc)
    depth = depthFinder(df)
    #Get Cost
    cost = 0
    for i in df['Gate']:
        cost += int(gateCosts[Gates.index(i)])
        
    return cost, depth

def binary_search(cost, name):  
    low = 0  
    high = int((np.math.factorial(4 + name - 1))/(np.math.factorial(name)*(np.math.factorial(3)))) - 1
    mid = 0  
  
    while low <= high:  
        # for get integer result   
        mid = (high + low) // 2  
        # print("Name", name)
        try:
            dfFull = pd.read_csv("D:/Documents/Combinations/combo" + str(name) + ".csv", skiprows = mid, nrows=1)
        except:
            with open(r"ErrorLog.csv", 'w', newline='', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow([name])
                return -1
        # Check if n is present at mid   
        if dfFull.loc[0][0] < cost:  
            low = mid + 1  
  
        # If n is greater, compare to the right of mid   
    
        elif dfFull.loc[0][0] > cost:  
            high = mid - 1  
  
        # If n is smaller, compared to the left of mid  
        else:  
            return mid  
  
            # element was not present in the list, return -1  
    return -1  
# binary_search(25, 20)

def Average(lst):
    return sum(lst) / len(lst)

In [40]:
#Selector Function
def selector(qc, percent):
    #Find Cost and Depth of the given QC
    cost, depth = ComplexityFinder(qc)
    # print(cost, depth)
    
    #Read Data for optimization level selection
    dfFull = pd.read_csv(dataFile)

    for i in range(len(dfFull)):
        refCost = dfFull.loc[i, "cost"]
        refDepth = dfFull.loc[i, "depth"]
        if refCost == cost and refDepth == depth:
            rowSet = i
            break
        else:
            rowSet = 0

    df = pd.read_csv(dataFile, skiprows = rowSet, nrows=1)

    df.columns = ['cost', 'depth', 'tqc3TT', 'tqc3RT', 'tqc3Depth', 'tqc2TT', 'tqc2RT', 'tqc2Depth', 'tqc1TT', 'tqc1RT', 'tqc1Depth', 'tqc0TT', 'tqc0RT', 'tqc0Depth']
    # print(df)
    #Get runtime of each optimization level for this qc
    tqc3RT = float(df["tqc3RT"]) + float(df["tqc3TT"])
    tqc2RT = float(df["tqc2RT"]) + float(df["tqc2TT"])
    tqc1RT = float(df["tqc1RT"]) + float(df["tqc1TT"])
    tqc0RT = float(df["tqc0RT"]) + float(df["tqc0TT"])
    RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]
    best = RTs.index(min(tqc3RT, tqc2RT, tqc1RT, tqc0RT))
    # print(RTs)

    for i in range(len(RTs)):
        if i != best:
            if abs(float(df["tqc" + str(i) + "Depth"]) - float(df["tqc" + str(best) + "Depth"])) / (float(df["tqc" + str(best) + "Depth"])) *100 <= percent and float(RTs[i]) < float(RTs[best]):
                best = i
        # print(df["tqc" + str(i) + "Depth"])
    return best

### Circuit Generation

In [44]:
runs = 10
maxComboLength = 290

for percent in range(0, 101, 10):
    simVal = []
    controlVal = []
    for cost in range(1, 30):
        #Cost has to be less than maxComboLength
        #Can change cost in intervals to decrease time
        #find all combinations of gates to = cost, include entanglement as soon as cost reaches large enough number
        #max depth == cost
        if cost >= maxComboLength:
            break
        for depth in range(1, cost):
            # clear_output(wait=True)
            #Define the combo length to look for
            name = random.randint(depth, depth * qubitCount)
            while name >= maxComboLength:
                name -= 1
            idx = binary_search(cost, name)
            # print("idx", idx)
            # print("Cost:", cost)
            # print("Depth:", depth)
            # # print("Name:", name)
            combination = []
            if int(idx) == -1:
                for t in range(depth, (depth * qubitCount)):
                    idx = binary_search(cost, t)
                    combination = []
                    if int(idx) != -1:
                        df = pd.read_csv(r"D:/Documents/Combinations/combo" + str(t) + ".csv", skiprows = idx, nrows=1)
                        for i in df.loc[0]:
                            if i in Gates:
                                combination.append(i)
                    if combination != []:
                        break
            else:  
                df = pd.read_csv(r"D:/Documents/Combinations/combo" + str(name) + ".csv", skiprows = idx, nrows=1)
                for i in df.loc[0]:
                    if i in Gates:
                            combination.append(i)
            qc = QuantumCircuit(qubitCount)
            listToAppend = []
            track = []
            if combination != []:
                comboTemp = combination
                random.shuffle(comboTemp)
                combo = []
                #Replace all x gates with x, y, or z
                for gate in comboTemp:
                    if gate == 'x':
                        combo.append(Gates[random.randint(0,2)])
                    else:
                        combo.append(gate)
                #append all to first qubit until depth is met
                #the rest are distributed randomly to the others 
                for gateIdx in range(len(combo)):
                    if gateIdx < depth:
                        if combo[gateIdx] == 'cx' or combo[gateIdx] == 'swap':
                            randint = random.randint(1, qubitCount - 1)
                            getattr(qc, combo[gateIdx])(0, randint)
                        else:
                            getattr(qc, combo[gateIdx])(0)
                    else:
                        if combo[gateIdx] == 'cx' or combo[gateIdx] == 'swap':
                            randint1 = random.randint(1, qubitCount - 1)
                            randint2 = random.randint(1, qubitCount - 1)
                            while True:
                                if track.count(randint1) < depth:
                                    while randint2 == randint1:
                                        randint2 = random.randint(1, qubitCount - 1)
                                    getattr(qc, combo[gateIdx])(randint1, randint2)
                                    track.append(randint1)
                                    break
                                else:
                                    randint1 = random.randint(1, qubitCount - 1)
                        else:
                            randint1 = random.randint(1, qubitCount - 1)
                            while True:
                                if track.count(randint1) < depth:
                                    getattr(qc, combo[gateIdx])(randint1)
                                    track.append(randint1)
                                    break
                                else:
                                    randint1 = random.randint(1, qubitCount - 1)
                qc.measure_all()

                testBest = selector(qc, percent)
                # print("Chosen Simulated Optimization Level: ", testBest)
                simVal.append(testBest)

                # Simulator CODE
                RTTimes = []
                TTTimes = []
                depths = []

                tempRT = []
                tempTT = []
                tempD = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 0)
                    tempTT.append(time.time() - start_time)
                    df = QCtoDF(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
                    tempRT.append(result.time_taken)
                    tempD.append(depthFinder(df))
                RTTimes.append(Average(tempRT))
                TTTimes.append(Average(tempTT))
                depths.append(Average(tempD))


                tempRT = []
                tempTT = []
                tempD = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 1)
                    tempTT.append(time.time() - start_time)
                    df = QCtoDF(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
                    tempRT.append(result.time_taken)
                    tempD.append(depthFinder(df))
                RTTimes.append(Average(tempRT))
                TTTimes.append(Average(tempTT))
                depths.append(Average(tempD))

                tempRT = []
                tempTT = []
                tempD = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 2)
                    tempTT.append(time.time() - start_time)
                    df = QCtoDF(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
                    tempRT.append(result.time_taken)
                    tempD.append(depthFinder(df))
                RTTimes.append(Average(tempRT))
                TTTimes.append(Average(tempTT))
                depths.append(Average(tempD))

                tempRT = []
                tempTT = []
                tempD = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 3)
                    tempTT.append(time.time() - start_time)
                    df = QCtoDF(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
                    tempRT.append(result.time_taken)
                    tempD.append(depthFinder(df))
                RTTimes.append(Average(tempRT))
                TTTimes.append(Average(tempTT))
                depths.append(Average(tempD))


                tqc3RT = float(RTTimes[3]) + float(TTTimes[3])
                tqc2RT = float(RTTimes[2]) + float(TTTimes[2])
                tqc1RT = float(RTTimes[1]) + float(TTTimes[1])
                tqc0RT = float(RTTimes[0]) + float(TTTimes[0])
                RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]
                best = RTs.index(min(tqc3RT, tqc2RT, tqc1RT, tqc0RT))


                # print("Simulated Times: ", RTs)
                # print("Simulated Depths: ", depths)


                for i in range(len(RTs)):
                    if i != best:
                        if abs(float(depths[i]) - float(depths[best])) / (float(depths[best])) *100 <= percent and float(RTs[i]) < float(RTs[best]):
                                best = i
                # print("Chosen Control Optimization Level: ", best)
                controlVal.append(best)
                
    #Get Accuracy
    #simVal is the array of selected optimization levels by my Selector function
    #controlVal is the array of selected optimization levels by the control
    correct = 0
    for i in range(len(simVal)):
        if simVal[i] == controlVal[i]:
            correct += 1

    print("Accuracy for percentage ", percent, " is: ", 100* (correct/(len(simVal)))) 

ZeroDivisionError: float division by zero

### Get Accuracy

In [22]:
#Get Accuracy
#simVal is the array of selected optimization levels by my Selector function
#controlVal is the array of selected optimization levels by the control
correct = 0
for i in range(len(simVal)):
    if simVal[i] == controlVal[i]:
        correct += 1
        
print("Accuracy: ", 100* (correct/(len(simVal)))) 

Accuracy:  45.812807881773395
